In [20]:
from pyspark.sql import SparkSession, functions as F

In [9]:
spark = SparkSession.builder\
  .master("local[*]")\
  .config("spark.driver.bindAddress", "127.0.0.1")\
  .appName("test")\
  .getOrCreate()

In [10]:
df_green = spark.read.parquet("./data/pq/green/*/*")

In [14]:
df_green = df_green\
  .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")\
  .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

In [11]:
df_yellow = spark.read.parquet("./data/pq/yellow/*/*")

In [15]:
df_yellow = df_yellow\
  .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")\
  .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")

In [18]:
shared_columns = [col for col in df_green.columns if col in df_yellow.columns]

In [22]:
df_green_sel = df_green.select(shared_columns).withColumn("service_type", F.lit("green"))
df_yellow_sel = df_yellow.select(shared_columns).withColumn("service_type", F.lit("yellow"))

In [23]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [24]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [26]:
df_trips_data.createOrReplaceTempView('trips_data')

In [27]:
spark.sql("""
  SELECT * FROM trips_data LIMIT 10;
""").show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2|2020-01-11 04:05:54|2020-01-11 04:13:49|                 N|       1.0|         129|         129|            1.0|         0.81|        6.5|  0.5|    0.5|      0.71|         0.0|       

In [28]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [29]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')

In [ ]:
spark.stop()